In [30]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

import MathOptInterface as MOI
import Ipopt 
import ForwardDiff as FD 
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc 
using Printf

include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

  Activating project at `~/Research/code/DroneTransportation/notebook`


animate_quadrotor (generic function with 1 method)

# Problem Setting

## Cost

In [39]:
function task_cost(params::NamedTuple, Z::Vector)::Real
    # compute the cost 
    xend = Z[params.idx.x[end]]
    J = (xend - params.xf)'*params.Q*(xend - params.xf)
    for k = 1:(params.N-1)
        x = Z[params.idx.x[k]]
        u = Z[params.idx.u[k]]
        J += (x - params.xf)'*params.Q*(x - params.xf) + u'*params.R*u
    end
    return J
end

task_cost (generic function with 1 method)

## Equality constraint

In [40]:
function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    x_mid =
        0.5 * (x1 + x2) +
        0.125 * dt * (quadrotor_dynamics(params.model, x1, u) - quadrotor_dynamics(params.model, x2, u))
    return x1 +
        1 / 6 *
        dt *
        (
            quadrotor_dynamics(params.model, x1, u) +
            4 * quadrotor_dynamics(params.model, x_mid, u) +
            quadrotor_dynamics(params.model, x2, u)
        ) - x2
end

function eq_constraints(params::NamedTuple, Z::Vector)::Vector
    c = zeros(eltype(Z), 12*(params.N+1))
    
    # dynamic constrains
    for i = 1:(params.N-1)
        xi = Z[params.idx.x[i]]
        xip1 = Z[params.idx.x[i+1]]
        ui = Z[params.idx.u[i]]        
        # dynamics constraints
        c[12*(i+1) .+ (1:12)] = hermite_simpson(params, xi, xip1, ui, params.model.dt)
    end

    # initial condition
    x1 = Z[params.idx.x[1]]
    c[1:12] = x1 - params.x0

    # final condition
    xf = Z[params.idx.x[end]]
    c[13:24] = xf - params.xf
    
    return c
end

eq_constraints (generic function with 1 method)

## Inequality constraint

In [41]:
function ineq_constraints(params::NamedTuple, Z::Vector)::Vector
    c = zeros(eltype(Z), 4*(params.N-1))
    for i = 1:(params.N-1)
        u = Z[params.idx.u[i]]
        c[4*(i-1) .+ (1:4)] = u
    end
    return c
end

ineq_constraints (generic function with 1 method)

## Task

In [46]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

"""
    quadrotor navigation

Function for returning collision free trajectories for 3 quadrotors. 

Outputs:
    x::Vector{Vector}  # state trajectory for quad
    u::Vector{Vector}  # control trajectory for quad
    t_vec::Vector
    params::NamedTuple

The resulting trajectories should have dt=0.2, tf = 5.0, N = 26
where all the x's are length 26, and the u's are length 25. 

"""
function quadrotor_navigation(;verbose=true)
    
    # problem size 
    nx = 12
    nu = 4
    dt = 0.05
    tf = 5.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial conditions and goal states 
    x0 = zeros(12)
    xf = zeros(12)
    xf[1] = 5.0
    
    # load all useful things into params 
    Q = 1.0 * diagm([1.0 * ones(3); 0.1 * ones(3); 1.0 * ones(3); 1.0 * ones(3)])
    R = 0.1 * I(nu)
    model = (mass=0.5,
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0,0,-9.81],
        L=0.1750, # drone arm length
        kf=1.0, u_max = 10.0/4.0, 
        km=0.0245,dt = dt)
    params = (
        N = N, 
        nx = nx, 
        nu = nu, 
        Q = Q, 
        R = R, 
        model = model, 
        x0 = x0,
        xf = xf,
        idx = idx
    )
    
    x_guess = range(params.x0, params.xf, length=params.N)
    z0 = zeros(params.idx.nz)
    for i = 1:params.N
        z0[params.idx.x[i][1]] = x_guess[i][1]
    end

    diff_type = :auto

    Z = fmincon(
        task_cost, 
        eq_constraints,
        ineq_constraints, 
        ones(params.idx.nz) .* -10,
        ones(params.idx.nz) .* 10,
        ones(4*(N-1)) .* 0.0, 
        ones(4*(N-1)) .* params.model.u_max,
        z0, 
        params,
        diff_type; 
        tol = 1e-6,
        c_tol = 1e-6,
        max_iters=10_000, 
        verbose = true
    )
    
    # return the trajectories 
    xs = [zeros(12)  for _ = 1:params.N]
    us = [zeros(4)  for _ = 1:(params.N-1)]
    for i = 1:params.N
        x = Z[params.idx.x[i]]
        xs[i] = x[1:12]
        if i < params.N
            us = Z[params.idx.u[i]]
        end
    end
        
    return xs, us, t_vec, params 
end

quadrotor_navigation

In [59]:
xs, us, t_vec, params = quadrotor_navigation()
display(animate_quadrotor(xs, xs, params.model.dt))

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------


---------successfully compiled both derivatives-----


---------IPOPT beginning solve----------------------


This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:  1973088
Number of nonzeros in inequality constraint Jacobian.:   644800
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1612
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1612
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1224
Total number of inequality constraints...............:      400
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      400
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.3835000e+01 4.91e-01 1.90e-13   0.0 0.00e+00    -  0.00e+00 0.00e+00 

┌ Info: Listening on: 127.0.0.1:8703, thread id: 1
└ @ HTTP.Servers /home/pcy/.julia/packages/HTTP/1EWL3/src/Servers.jl:369
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/pcy/.julia/packages/MeshCat/I6NTX/src/visualizer.jl:63


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("traj" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("p94" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x39, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s14" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x31, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s44" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x34, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s52" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x35, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p92" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x39, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p98" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x39, 0x38], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p6" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x61, 0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x36], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p23" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x32, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p90" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x39, 0x30], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s94" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x39, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}())…)), "vic" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x76, 0x69, 0x63], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "target" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x74, 0x2f, 0x74, 0x61, 0x72, 0x67, 0x65, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8703))

In [58]:
xs = hcat(xs...)
us = hcat(us...)
plot(t_vec, xs[1:3], title = "Position", label = ["x" "y" "z"], xlabel = "Time [s]", ylabel = "Position [m]")
display(plot!(t_vec, xs[4:6], title = "Velocity", label = ["vx" "vy" "vz"], xlabel = "Time [s]", ylabel = "Velocity [m/s]"))